In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment

In [2]:
def cria_chroma_from_sample(sample, sample_rate):

    y_harmonic = librosa.effects.harmonic(sample)

    # Compute chroma features from the harmonic signal
    # Transforma cada faixa de frequencia em notas musicais
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic,
                                            sr=sample_rate)
    
    return chromagram

In [3]:
def identifica_picos_12(lista):

    n1, n2, n3 = -1, 0, 1
    picos_notas = []
    picos_notas

    # verifica de 3 em 3, procurando picos
    while n2 < len(lista):
        if( lista[n1] < lista[n2] > lista[n3]) and 2 *lista[n2] - lista[n1] -lista[n3] > 1:
            picos_notas.append([n2, lista[n2]])

        n1, n2, n3 = n1+1, n2+1, n3+1

        n3 = n3%12

    # ordena com base no volume
    picos_notas = sorted(picos_notas, key=lambda x: x[1], reverse=True)
    picos_notas = [linha[0] for linha in picos_notas]


    return picos_notas


In [4]:
def identifica_fundamental(chroma):

    #critério de seleção: linha com o maior valor
    linhas_somadas = np.sum(chroma, axis=1)

    # argsort() retorna os índices que ordenariam um array.
    # [-2:] pega os dois últimos índices, que correspondem às maiores somas
    picos = identifica_picos_12(linhas_somadas)

    # pega uma nota só
    #indice_nota = np.argmax(linhas_somadas) 

    tam_picos = len(picos)
    maior_menor = False
    print(picos)

    # se não tiver notas, ignora
    if tam_picos == 0:
        return False
    # se o array só tiver uma nota, ela vai ser a tonica
    elif tam_picos == 1:
        indice_nota = picos[0]
    # se o array tiver 2 ou 3 notas
    elif tam_picos == 2:

        #identifica a quinta
        if (picos[0] + 7 ) %12 == picos[1]:
            indice_nota = picos[0]
        elif (picos[1] + 7 ) %12 == picos[0]:
            indice_nota = picos[1]
        else:
            indice_nota = picos[0]

    # array com 3 ou mais notas    
    elif tam_picos > 2:
        tem_quinta = False

        for i in picos:
            quinta = (i+7) % 12
            
            if quinta in picos:

                tem_quinta = True
                indice_nota = i
                picos.remove(i)
                picos.remove(quinta)

                terca_menor = (i+3) % 12
                terca_maior = (i+4) % 12

                # busca pela terca
                if terca_menor in picos:
                    maior_menor = "menor" # menor
                elif terca_maior in picos:
                    maior_menor = "maior" #maior
                else:
                    maior_menor = picos
                break
        
        if  not tem_quinta:
            indice_nota = picos[0]
            print("ERRO nao tem quinta")
        

    notas_musicais = {
    0: "Dó",
    1: "Dó#",
    2: "Ré",
    3: "Ré#",
    4: "Mi",
    5: "Fá",
    6: "Fá#",
    7: "Sol",
    8: "Sol#",
    9: "Lá",
    10: "Lá#",
    11: "Si"
    }

    nota_fundamental = notas_musicais[indice_nota]
    
    return nota_fundamental, maior_menor

In [11]:
def final_process(file = str, format = str):
    # Carregar o arquivo mp3
    audio = AudioSegment.from_file(file, format=format)

    # Definindo a duração de cada segmento em milissegundos
    #duracao_segmento = 167  # 1/6 segundos
    duracao_segmento = 500  # 1/6 segundos

    notas_fundamentais_acordes = []

    # Segmentando o áudio
    for i in range(0, len(audio), duracao_segmento):
        segmento = audio[i:i + duracao_segmento]
        print(i, i+duracao_segmento)

        # transforma audio no formato que o librosa trabalha
        sample = np.array(segmento.get_array_of_samples()).astype(np.float32) / 32768
        sample_rate = segmento.frame_rate

        chromagram = cria_chroma_from_sample(sample=sample, sample_rate=sample_rate)

        fundamental = identifica_fundamental(chroma=chromagram)

        notas_fundamentais_acordes.append(fundamental)

    
    #notas_fundamentais_acordes = contar_sequencias_notas(notas_fundamentais_acordes)
    return(notas_fundamentais_acordes)


In [12]:
notas_final = final_process('felipelima.m4a', 'm4a')

0 500
[7, 10, 2, 4]
500 1000


c:\Users\felip\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=1000
  warnings.warn(
c:\Users\felip\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=500
  warnings.warn(
c:\Users\felip\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=1024 is too large for input signal of length=250
  warnings.warn(


[7, 9, 2, 0, 4]
1000 1500
[6, 10, 3]
1500 2000
[10, 5]
2000 2500
[10, 6, 2]
ERRO nao tem quinta
2500 3000
[10, 6, 3]
3000 3500
[10, 6, 4]
ERRO nao tem quinta
3500 4000
[7, 11, 2]
4000 4500
[7, 2, 11]
4500 5000
[7, 11, 2]
5000 5500
[7, 11]
5500 6000
[7, 11, 2]
6000 6500
[7, 11, 2]
6500 7000
[7, 11, 2]
7000 7500
[7, 11, 2]
7500 8000
[7, 11, 2]
8000 8500
[7, 11, 2]
8500 9000
[7, 11, 2]
9000 9500


KeyboardInterrupt: 

In [10]:
notas_final

[('Sol', 'menor'),
 ('Sol', [9, 0, 4]),
 ('Ré#', 'menor'),
 ('Lá#', False),
 ('Lá#', False),
 ('Ré#', 'menor'),
 ('Lá#', False),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', False),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', 'maior'),
 ('Sol', [9, 4]),
 ('Sol', [4, 0]),
 ('Dó', 'maior'),
 ('Dó', 'maior'),
 ('Dó', 'maior'),
 ('Dó', 'maior'),
 ('Sol', 'menor'),
 ('Sol', [4, 0]),
 ('Sol', [4, 0]),
 ('Sol', [4, 0]),
 ('Sol', [4, 0]),
 ('Sol', [4, 0]),
 ('Dó', 'maior'),
 ('Sol', [4, 0]),
 ('Sol', 'menor'),
 ('Dó', 'maior'),
 ('Sol', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá', 'menor'),
 ('Lá',